# Example: Fitting Broad components and Narrow-Line Region Emission Lines 

This notebook steps through using the BEAT routine to fit multiple components to [O III] 4959,5007 and H-beta emission lines in PG_1307+085 which also includes some broad components. Spectra for this notebook are included in the 'spectra_pg' folder.

In [1]:
import beat
import numpy as np
import os
import matplotlib.pyplot as plt

# Reading in data

Here we define a function (load_file) to read a spectrum .txt file and create arrays for wavelength, flux, and noise (i.e. error). In this example, the .txt files being read only have 3 columns of data - wavelength, flux, and noise.

We use numpy.loadtxt to read in the data, but one can use any method that returns the required wavelength, flux, and noise arrays. 

In [16]:
def load_file(filepath):
    """reads file and returns wave, flux, and noise arrays to pass to fitting code"""
    #wave, flux, noise = np.loadtxt(filepath, usecols=(0, 1, 2), unpack=True, delimiter='\t', skiprows=1)
    wave, flux, noise = np.loadtxt(filepath, usecols=(0, 1, 2), unpack=True, delimiter=',', skiprows=0)
    return wave, flux, noise

# Fitting a spectrum

Run the next two cells without changing anything. This will run BEAT and produce an output folder, in this case called `ngc7319_out`. If BEAT ran succesffully, the `plots` folder within `ngc7319_out` will show three plot files for zero-, one-, and two-component fits.

A description of the parameters in the `main()` function are on the docs page (link), although the docs page and more advanced Jupyter notebooks will have more details.

In [17]:
def main():
    target_param = {'name': 'ngc7319',      # Name of target
                    'red': 0.022,    # Redshift of target
                    #'minwidth': 3.83, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'minwidth': 1.5, #2.16, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 5,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                    'start': 3180,       # Minimum array value of used spectrum [array units]
                    #'end': 1100,        # STIS Maximum array value of used spectrum [array units]
                    'end': 3910,        # APO Maximum array value of used spectrum [array units]
                    'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 6400,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 6900,#6725,   # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 2,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 3}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (6400, 6450), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                         #'continuum2': (6775, 6825)
                        'continuum2': (6800, 6850)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]

                        
    fit_instructions = {
                        'line1': # Instructions for narrow line component OIII (5008)
                                {'name': 'halpha', 
                                 'wave': 6562.8, 
                                 'minwave': 6700, #should be redshifted version
                                 'wave_range': 40.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                         'line2': # Instructions for narrow line component H-Beta (4861)
                                {'name': 'nii', 
                                 'wave': 6583.46, 
                                 #'minwave': 6583.46,#*(1+ 0.003859) - 26, 
                                 #'wave_range': 52.0, 
                                 'flux_free': True},       # Allow flux to vary freely
                        'line3': # Instructions for narrow line component OIII (4960)
                                {'name': 'nii', 
                                  'wave': 6548.04, 
                                  'flux_free': False,     # Lock flux with another component
                                  'locked_with': 'line2', # Lock flux w/ 'line1' (OIII 5008)
                                  'flux_ratio': 3},       # Flux ratio between line2/line3 = 3
                       
    }
    
    fit = beat.Fit(out_dir='', #beat_fits
                   spec_dir='spectrum',
                   load_file=load_file,
                   target_param=target_param,
                   cont_instructions=cont_instructions,
                   fit_instructions=fit_instructions,
                   )
    fit.mp_handler()

In [18]:
main()

remaining to fit: 2
Fitting NGC7319.csv
Min Y: 3.48e-16
Max Y: 2.22e-15
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    1
 *****************************************************
 ln(ev)=  -336.53376355324764      +/-   5.0604308311441246E-007
 Total Likelihood Evaluations:          200
 Sampling finished. Exiting MultiNest
components:  0
NGC7319.csv: trying 1 component(s)
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    4
 *****************************************************
 ln(ev)=  -98.136213991573541      +/-  0.23871108162676227     
 Total Likelihood Evaluations:         5566
 Sampling finished. Exiting MultiNest
components:  1
broad profile:  [6.40677243e+03 1.00700488e+02 1.50608395e-14]
NGC7319.csv: trying 2 component(s)
 **

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte